In [1]:
%load_ext autoreload
%autoreload 2

# general
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.interpolate import interp1d

# plotting
import matplotlib.pyplot as plt
from ipywidgets import interact
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
from plotly.subplots import make_subplots
import plotly.graph_objs as go

# fitting
from sklearn.metrics import r2_score
from scipy.optimize import minimize
from functools import partial
import multiprocess as mp
import matplotlib.patches as mpatches

# custom
from reflectance import plotting, spectrum_utils, optimisation_pipeline, file_ops

# Pipeline

In [5]:
config_dict = {"processing": {
    'aop_group_num': 1,
    'nir_wavelengths': [750, 1100],
    'sensor_range': [450, 690],
    'endmember_type': 'mean',
    'endmember_normalisation': False,
    'endmember_class_schema': 'three_endmember',
    'spectra_normalisation': False},
                 "simulation":
{    "type": "spread",
    "N": 10,
    "Rb_vals": [0,1,0],
    "n_depths": 10,
    "depth_lims": [0, 10],
    "n_ks": 10,
    "k_lims": [0, 0.4],
    "n_bbs": 10,
    "bb_lims": [0.01, 0.03],
    "n_noise_levels": 10,
    "noise_lims": [0.001, 0],
    "noise_ind": 0}, # TODO: remove unnecessary noise vars
    "fitting":{
    'objective_fn': 'r2',
    'bb_bounds': (0, 0.41123),
    'Kd_bounds': (0.01688, 3.17231),
    'H_bounds': (0, 40),
    'solver': 'CG',
    'tol': 1e-6
}}
glob_cfg, _ = file_ops.instantiate_single_configs_instance(run_ind = 0)

cfg = file_ops.RunOptPipeConfig(config_dict)
g_cfg = glob_cfg

pipe = optimisation_pipeline.OptPipe(g_cfg, cfg)
pipe.run()

Generating simulated spectra:   0%|          | 0/10 [00:00<?, ?it/s]

Fitting spectra:   0%|          | 0/100 [00:00<?, ?it/s]

RunOptPipeConfig(aop_group_num=1, nir_wavelengths=[750, 1100], sensor_range=[450, 690], endmember_type='mean', endmember_normalisation=False, endmember_class_schema='three_endmember', spectra_normalisation=False, objective_fn='r2', bb_bounds=(0, 0.41123), Kd_bounds=(0.01688, 3.17231), H_bounds=(0, 40), simulation={'type': 'spread', 'N': 10, 'Rb_vals': [0, 1, 0], 'n_depths': 10, 'depth_lims': [0, 10], 'n_ks': 10, 'k_lims': [0, 0.4], 'n_bbs': 10, 'bb_lims': [0.01, 0.03], 'n_noise_levels': 10, 'noise_lims': [0.001, 0], 'noise_ind': 0}, solver='CG', tol=1e-06)


In [ ]:
# view fits
test_spectra = pd.read_csv(file_ops.RESULTS_DIR_FP / "fits/sim_spectra_1.csv")
wvs = test_spectra.iloc[:,3:].columns.astype(float)
plt.plot(wvs, test_spectra.iloc[:,3:].values.T, color='k', alpha=0.01);

In [ ]:
test_fits = pd.read_csv(file_ops.RESULTS_DIR_FP / "fits/fit_results_nan.csv", skiprows=1)
plt.plot(wvs, test_fits.iloc[:,6:-2].values.T, color='k', alpha=0.01);

In [ ]:
# plot r2
plt.hist(test_fits.r2, bins=50);

In [ ]:
# plot fitted spectra


In [ ]:
fits.iloc[:12,:3]

In [ ]:
fits

In [ ]:
# three_endmember_array.shape
wvs.shape

In [ ]:
out, _ = spectrum_utils.spread_simulate_spectra(wvs, three_endmember_array, AOP_args, N=200, Rb_vals=(0.4, 0.4, 0.2))
plt.plot(out.reshape(-1, out.shape[-1]).T, color='k', alpha=0.01);

In [ ]:
glob_cfg, run_cfg = file_ops.instantiate_single_configs_instance(run_ind = 0)

pipe = optimisation_pipeline.OptPipe(glob_cfg, run_cfg)
pipe.run()

In [5]:
config_params_dict = {
    'processing': {
        'aop_group_num': [1, 2, 3],
        'nir_wavelengths': [[750, 1100]],
        'sensor_range': [[450, 690]],
        'endmember_type': ["mean", "median", "pca", "nmf", "ica", "svd"],
        'endmember_normalisation': ["minmax", "zscore", "robust", "maxabs"],
        'endmember_class_schema': ["four_endmember", "all", "inorganic_organic"],
        'spectra_normalisation': ["minmax", "zscore", "robust", "maxabs"]
    },
    "simulation": {
        "type": ["spread"],
        "N": [100],
        "Rb_vals": [(0,1)],
        "n_depths": [10],
        "depth_lims": [(0, 10)],
        "n_ks": [10],
        "k_lims": [(0, 0.4)],
        "n_bbs": [10],
        "bb_lims": [(0.01, 0.03)],
        "n_noise_levels": [10],
        "noise_lims": [(0.001, 0)],
        "noise_ind": [0] 
    },
    'fitting': {
        'objective_fn': ["r2", "spectral_angle"],
        'bb_bounds': [(0, 0.41123), (0, 0.2)],
        'Kd_bounds': [(0.01688, 3.17231), (0.01688, 1.5)],
        'H_bounds': [(0, 10), (0, 40), 
                    #  (0, 20), (0, 30)
                     ],
        'solver': ['L-BFGS-B'], 
        # 'Powell', 'CG', 'BFGS', 'Newton-CG', 'L-BFGS-B', 'TNC', 'COBYLA', 'COBYQA', 'SLSQP', 'trust-constr', 'dogleg', 'trust-ncg', 'trust-exact', 'trust-krylov'],
        'tol': [
            # 0.0, 1e-3, 
            1e-6, 1e-9]
    }
}


run_cfgs = file_ops.generate_config_dicts(config_params_dict)
glob_cfg = file_ops.read_yaml(file_ops.CONFIG_DIR_FP / "glob_cfg.yaml")
exec_kwargs = {
    "tqdm": False,
    "n_cpus": 128
}

In [ ]:
run_cfgs[0]

In [ ]:
optimisation_pipeline.run_pipeline(glob_cfg, run_cfgs[:3])